# Implementation of Draper's QFT adders

### By Mathis Beaudoin (Summer 2024)

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from shors_algorithm.methods.utils.draper import phi_add, c_phi_add, cc_phi_add, q_adder
from shors_algorithm.methods.utils.QFT import build_QFT, build_QFT_dag
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

### Add classical integer to quantum integer

In [ ]:
c_integer = 2
q_integer = 3

num_qubits = 4
register = [i for i in range(num_qubits)]

basic_adder = QuantumCircuit(num_qubits)
basic_adder.initialize(f'{q_integer:b}'.zfill(num_qubits), register)

basic_adder.append(build_QFT(num_qubits), register)
phi_add(basic_adder, c_integer, register)    #inverse = True to subtract
basic_adder.append(build_QFT_dag(num_qubits), register)

result = Statevector(basic_adder).probabilities_dict(decimals=10)
bit_string = list(result.keys())

print("Phi_add result :", result)
print("Integer result :", int(bit_string[0],2))

### Control-add classical integer to quantum integer

In [ ]:
c_integer = 1
q_integer = 2

num_qubits = 5
register = [i for i in range(1, num_qubits)]
ctrl_qubit = 0 #We choose the ctrl qubit to be the first qubit in the circuit

qc_c_adder = QuantumCircuit(num_qubits)
qc_c_adder.x(ctrl_qubit)    #We activate the control qubit
qc_c_adder.initialize(f'{q_integer:b}'.zfill(num_qubits - 1), register)

qc_c_adder.append(build_QFT(num_qubits - 1), register)
c_phi_add(qc_c_adder, c_integer, ctrl_qubit, register)
qc_c_adder.append(build_QFT_dag(num_qubits - 1), register)

result = Statevector(qc_c_adder).probabilities_dict(decimals=10)
bit_string = list(result.keys())

print("c_phi_add result :", result)
print("Integer result : ", int(bit_string[0][:-1],2))

### Control-control-add classical integer to quantum integer

In [ ]:
c_integer = 1
q_integer = 2

num_qubits = 5
register = [i for i in range(2, num_qubits)]
ctrl_qubit_1 = 0
ctrl_qubit_2 = 1

cc_add = QuantumCircuit(num_qubits)
cc_add.x([ctrl_qubit_1, ctrl_qubit_2])
cc_add.initialize(f'{q_integer:b}'.zfill(num_qubits - 2), register)

cc_add.append(build_QFT(num_qubits - 2), register)
cc_phi_add(cc_add, c_integer, ctrl_qubit_1, ctrl_qubit_2, register) #inverse = True to subtract
cc_add.append(build_QFT_dag(num_qubits - 2), register)

result = Statevector(cc_add).probabilities_dict(decimals=10)
bit_string = list(result.keys())

print("cc_phi_add result :", result)
print("Integer result : ", int(bit_string[0][:-2],2))

### Add two quantum integers

In [ ]:
q_integer_1 = 2 
q_integer_2 = 2

num_qubits = 3 #number of qubits for each quantum register
register_1 = [i for i in range(num_qubits)] #A register for each quantum number
register_2 = [i for i in range(num_qubits, 2*num_qubits)] #Result will be in this register

qc_q_adder = QuantumCircuit(2*num_qubits)
qc_q_adder.initialize(f'{q_integer_1:b}'.zfill(num_qubits), register_1)
qc_q_adder.initialize(f'{q_integer_2:b}'.zfill(num_qubits), register_2)

qc_q_adder.append(build_QFT(num_qubits), register_2)
q_adder(qc_q_adder, register_1, register_2)
qc_q_adder.append(build_QFT_dag(num_qubits), register_2)

result = Statevector(qc_q_adder).probabilities_dict(decimals=10)
bit_string = list(result.keys())

print("Quantum adder result :", result)
print("Integer result : ", int(bit_string[0][0:num_qubits],2))